In [ ]:
#setup
wd = '/course/popgen23/ben/fstats_tutorial/software'
dd = '/course/popgen23/ben/fstats_tutorial'
setwd(wd)
options(repr.plot.width=22, repr.plot.height=22)
suppressPackageStartupMessages({
    library(admixtools)                                                  
    library(ape)                                                         
    library(tidyverse)  
    library(gplots)
    library(glue)
    source(glue("{wd}/scripts//analysis.R"))
    source(glue("{wd}/scripts//treemix_plotting2.R"))
})

## 1. Estimating trees
Many methods to visualize population structure between many populations can be written as functions of a _distance matrix_, like a matrix of $F_2$ or $F_{ST}$-values. These methods include principal component analysis, trees and admixture graphs, the latter two we will focus on today. In distance matrices, large values mean that samples are less similar to each other, in contrast to similarity matrices where large values imply samples are _more_ alike.

In [ ]:
#here we read all possible f2 values
f2s = read_f2(glue('{dd}/fdata/worldfoci2'))

#we arrange them into a matrix
f2_mat = f2_matrix(f2s)

In [ ]:
heatmap.2(f2_mat, symm=T, scale="none", trace='n')

**Task 1:** Use the distance matrix in the `f2_mat`-matrix to generate a tree using the neighbor joining algorithm.
Use the following functions from the `ape` package:
 
 - `nj()` to calculate the tree
 - `root()` to set an appropriate root
 - `plot.phylo()` to visualize your data

In case you are not sure how these functions work, the help (typing e.g. `?nj` may help)

**Questions:**
   - Can you tell which populations might be admixed?
   - how does that relate to $f_3$-statistics?

In [ ]:
#Type your solution here

## 2. Fitting admixture graphs using qpgraph
Our next task will be to use `qpgraph` to fit a graph. In contrast to `nj`, `qpgraph` does not estimate a graph, so one has to be specified. It will then fit all admixture proportions and branch lengths. One can think of this as obtaining "plausible" graphs that fit the data, without any statements whether they are the best or unique graph for that purpose. Thus, we have to specify a graph. The easiest way is to give it a list using 2 columns, where each row is anedge in the graph.

In [ ]:
starting_graph = rbind(                   
              c("ROOT", "Mbuti"),       
              c("ROOT", "AFR"),              
              c("AFR", "AFR1"),           
              c("AFR1", "AA"),     
              c("AFR", "OOA"),            
              c("OOA", "EUR"),            
              c("OOA", "EAS"),            
              c("EUR", "Sardinian"),         
              c("EUR","AA"),       
              c("EAS", "Han")             
              )                             

For this purpose, we have to define a bunch of internal nodes (a useful convention is to use ALL-CAPS for internal nodes). I.e. in our starting graph, we start at the root, where we us the _Mbuti_ as an outgroup. Within Africa, we have two populations; the African component of the African-American population (_AA_), and the out-of-Africa population (OOA). Outside Africa, we split into Europe (EUR) and East Asians (EAS), represented by the _Sardinian_ and _Han_, respectively. Our hypothesis is that the African-Americans have some European ancestry, therefore we add a second edge from `EUR` to `AA` to reflect that.

In [ ]:
#fitting the graph is done using the qpgraph function:
G = qpgraph(f2s,starting_graph, return_fstats = T); 
plot_graph(G$edges, textsize=10)

Next, we can look at the fitted $f_2$-statistics

In [ ]:
G$f2

the columns
- `est` is the estimated f-statistic
- `se` is the estimated standard error
- `fit` is the f3/f4-statistic calculated under the model
- `diff` and `z` give the difference `fit-est` and associated Z-score, respectively.

If there are no Z-scores above 3 (or below -3), the model is often considered a fit to the data (again, with no claim about whether it is the best model). In this simple example, we only have 6 $f_2$-values and estimate 6 parameters (5 branch lengths and one admixture coefficient), so it is not surprising that we obtain a perfect fit.

**Task 2:** Assume we want to compare this model where the `AA`-population only has African Ancestry. For this purpose, remove the edge from `EUR` to `AA` from the graph. Does this model also fit?


In [ ]:
# Type your solution here

**Task 3:** We picked `Sardinian` as a "proxy" for Europeans, and `Mbuti` as a proxy for Africans. What happens if we substitute them for other populations from the same region (e.g. `Sardinian -> Basque`, `Mbuti -> Yoruba`). What happens if we all add `Yoruba`, `Mbuti`, `Sardinian` and `Basque` to the model? 

In [ ]:
# Type your solution here

## 3. Estimating Admixture Graphs using Treemix
`qpgraph` has the drawback that the user needs to specify the graph. This has the advantage that we retain fine-scale control over the fitting, but in many cases we would like to have a way to estimate the tree from data. 

This is the problem addressed by `treemix` [(Pickrell & Pritchard 2012)](https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1002967). Unfortunately, the program has not been maintained for the last five+ years, so it is a little bit tedious to use. For this exercise, I converted the input files for the `world`-data set to treemix format (in `treemix/worldfoci2.treemix.in.gz`), and ran treemix with 0 to 5 admixture edges ((`-m 0` to `-m 5`), the resulting output are stored in the `treemix`-folder as well

In [ ]:
#system("treemix -i treemix/worldfoci2.treemix.in.gz -m 0 -k 1000 -root Mbuti -o treemix/worldfoci2.treemix.m0")
#system("treemix -i treemix/worldfoci2.treemix.in.gz -m 1 -k 1000 -root Mbuti -o treemix/worldfoci2.treemix.m1")
#system("treemix -i treemix/worldfoci2.treemix.in.gz -m 2 -k 1000 -root Mbuti -o treemix/worldfoci2.treemix.m2")
#system("treemix -i treemix/worldfoci2.treemix.in.gz -m 3 -k 1000 -root Mbuti -o treemix/worldfoci2.treemix.m3")
#system("treemix -i treemix/worldfoci2.treemix.in.gz -m 4 -k 1000 -root Mbuti -o treemix/worldfoci2.treemix.m4")
#system("treemix -i treemix/worldfoci2.treemix.in.gz -m 5 -k 1000 -root Mbuti -o treemix/worldfoci2.treemix.m5")

The output of `treemix` will be wo plots; one is the fitted graph, and one is a plot of the residuals that gives us some information about which pairs of populations are not fit well. We can use the `plot_treemix` and `plot_resid`-functions for this purpose: (these will produce some warnings, which we can ignore)

In [ ]:
tm = plot_treemix(glue('{dd}/treemix/worldfoci2.treemix.m0'))


In [ ]:
tree = nj(f2_mat)
res = plot_resid(glue('{dd}/treemix/worldfoci2.treemix.m0'), pop_order=get_tip_order(tree))

In [ ]:
#Type your commands here

**Task 4**: Have a look at the admixture graphs and residual plots for `m=0` to `m=5`. 

- Does the fit improve when adding more admixture events?
- Can you predict how the fit improves?
- Does `m=5` provide a good fit?

In [ ]:
# Add your code here

# Solutions
### Task 1

In [ ]:
f2_mat %>% nj %>% root('Mbuti') %>% plot

### Task 2

In [ ]:
graph_task2 = rbind(                   
              c("ROOT", "Mbuti"),       
              c("ROOT", "AFR"),              
              c("AFR", "AFR1"),           
              c("AFR1", "AA"),     
              c("AFR", "OOA"),            
              c("OOA", "EUR"),            
              c("OOA", "EAS"),            
              c("EUR", "Sardinian"),         
              #c("EUR","AA"),       
              c("EAS", "Han")             
              )  
G2 = qpgraph(f2s,graph_task2,return_fstats=T); 
plot_graph(G2$edges, textsize=10)

### Task 3

In [ ]:
graph_task3 = rbind(                   
              c("ROOT", "Mbuti"),       
              c("ROOT", "AFR"),              
              c("AFR", "AFR1"),           
              c("AFR1", "AA"),     
              c("AFR", "OOA"),            
              c("OOA", "EUR"),            
              c("OOA", "EAS"),            
              c("EUR", "Basque"),         
              #c("EUR","AA"),       
              c("EAS", "Han")             
              )                             
G3 = qpgraph(f2s,graph_task3,return_fstats=T); 
plot_graph(G3$edges, textsize=10)



### Task 4

In [ ]:
for(i in 0:5){
    plot_treemix(glue('{dd}/treemix/worldfoci2.treemix.m{i}'))
    plot_resid(glue('{dd}/treemix/worldfoci2.treemix.m{i}'), pop_order=get_tip_order(tree))
    }